#I/ TỔNG QUAN
File này trình bày về quá trình crawl data bằng Selenium từ website của Công ty Chứng Khoán Rồng Việt ([Link](https://data.vdsc.com.vn/)) để lấy số liệu tài chính của các công ty trong "**Danh sách Công ty niêm yết trên sàn HOSE**" đã crawl trước đó (tham khảo file ipybn "*A2_Scraping_P1*") và lưu thành các file csv riêng biệt (Vd: "*AAA.csv*", "*VCB.csv*" ...) trong folder "*bctc*" trong Collab.

Các file data raw này sau đó được xử lý và tổng hợp thành 1 DataFrame chứa chỉ số tài chính cho tất cả các Công ty và lưu thành 1 file csv tên là "*B2_finance_record*". File này là file đầu vào cho thuật toán Machine Learning trong Project.

> **LƯU Ý**:

- Trước khi chạy file này cần upload file "*B1_hse_comp.csv*" lên Google Collab.
- File có 2 phần riêng biệt có thể chạy **độc lập** là:
 1. Crawl data: Phần này chạy rất lâu (~ 4h) và **chỉ chạy được trên trình duyệt Chrome** cho kết quả là các file csv lưu trong folder "*bctc*" đã nói phía trên.
 2. Xử lý & tổng hợp data: Nếu chỉ chạy phần này mà không chạy phần trên thì cần upload các file csv vào folder "*bctc*" trên Google Collab.

# II/ IMPORT LIBRARY VÀ MODULE
Phần này import các library & module sử dụng cho 2 phần tiếp theo.

In [ ]:
# import các thư viện xử lý dataframe và số
import pandas as pd
import numpy as np

In [ ]:
# thiết lập tùy chọn thể hiện 2 số thập phân
pd.options.display.float_format = "{:.2f}".format

# lấy data các Công ty từ file file crawl trước đó
companies = pd.read_csv('B1_hse_comp.csv')

# tạo ds mã CK từ Ds các Công ty
mck_list = list(companies['Mã'])

#III/ CRAWL DATA VỚI SELENIUM
  Selenium là một bộ công cụ mã nguồn mở chuyên dùng để tự động hóa công tác kiểm thử phần mềm/website/browser.

  Đặc biệt là giúp mô phỏng hầu hết những thao tác người dùng với trình duyệt (nhấp - click, cuộn - scroll…) thông qua công cụ Selenium WebDriver.
  
  Từ đó chúng ta có thể sử dụng để tự động hóa việc scrape data từ các trang web dynamic.

## 1/ Cài đặt Selenium & webdriver manager

In [ ]:
# cài đặt selenium version 4.11
!pip install selenium==4.11.0

# cài đặt trình quản lý webdriver
!pip install webdriver-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00


## 2/ Thiết lập option & tạo browser Chrome

In [ ]:
# import trình chạy webriver và thiết lập tùy chọn
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# import module tạo số ngẫu nhiên và thực hiện tạm nghỉ
from time import sleep
from random import randint

# khởi tạo trình tùy chọn
options = Options()

# thêm tùy chọn
options.add_argument('--no-sandbox') # không tạo sandbox (môi trường cách ly dùng chạy thử & testing)
options.add_argument('--disable-gpu') # tắt GPU để tăng bộ nhớ RAM
options.add_argument('--headless=new') # điều khiển browser tương tác với web mà không cần mở xem giao diện web
options.add_argument('--disable-dev-shm-usage') # tắt shm để Chrome không bị fail or crash

# khởi tạo browser Chrome (chỉ chạy được trên trình duyệt Chrome)
browser = webdriver.Chrome(options=options)

## 3/ Tạo hàm xử lý

In [ ]:
# tạo hàm lấy bảng dữ liệu (các chỉ số tài chính) từ web RongViet
def get_table(mck):

  # tìm các row của bảng có id="contentQuarter"
  table_rows = browser.find_elements(By.XPATH,'//*[@id="contentQuarter"]/table/tbody/tr') # Cty Thuong & NGAN HANG Vd DGW, FPT, VIC

  # nếu như không tìm được (vì có 2 bảng với 2 id)
  if table_rows == []:

    # thì tìm với id="contentQuarter"
    table_rows = browser.find_elements(By.XPATH,'//*[@id="content_Quarter"]/table/tbody/tr') # Cty CK & BAO HIEM (tru PGI) Vd SSI, VDS, VND

    # nếu như vẫn không tìm được
    if table_rows == []:

      # báo mã bị lỗi để sau đó kiểm tra lại
      print(f'MA {mck} BI LOI !!!')

  # tạo list trống lưu dữ liệu các row
  rows_data = []

  # tạo loop xử lý từng row và lưu dữ liệu
  for row in table_rows:

    # tìm các column có trong row này
    columns = row.find_elements(By.XPATH,"./td") # đánh dấu '.' trước tag 'td' trong xpath de tim 'td' trong 'tr'

    # tạo biến lưu dữ liệu từng row
    row_data = []

    # tạo loop xử lý từng column
    for column in columns:

      # lấy dữ liệu từng column và lưu lại
      row_data.append(column.text)

    # lưu row này vào biến chứa dữ liệu các row ban đầu
    rows_data.append(row_data)

  # lưu dữ liệu vào 1 dataframe và bỏ cột null (la cot chua bieu do barchart trong bang tren website)
  df = pd.DataFrame(rows_data)
  df = df.drop(columns=1)

  # trả lại dataframe kết quả
  return(df)

In [ ]:
# tạo hàm lấy data nhiều bảng từ 1 trang web của 1 mã chứng khoán (CK)
def get_mck_csv(mck):

  # tạo địa chỉ url tương ứng với từng mã CK
  url = 'https://data.vdsc.com.vn/vi/Com_Report/' + mck + '/'

  # truy cập địa chỉ url này
  browser.get(url)

  # tạm nghỉ 3s để load trang
  sleep(3)

  # lấy bảng chỉ số tài chính lần 1 chứa thông tin của 4 Quý gần nhất
  df1 = get_table(mck)

  # tạo con trỏ tại vị trí lùi về sau của bảng nói trên
  back_cursor = browser.find_element(By.XPATH,'//*[@id="spNext1"]')

  # thực hiện click để lui về 4 Quý sau
  back_cursor.click()

  # tạm nghỉ 3s để load trang
  sleep(3)

  # lấy bảng chỉ số tài chính lần 2 của 4 Quý sau
  df2 = get_table(mck)

  # nối 2 bảng này lại theo thứ tự thời gian
  df = pd.merge(df2, df1, on=0)

  # tạo file path cho file csv sắp lưu
  csv_path = '/content/bctc/' + mck + '.csv'

  # lưu kết quả vào file csv trong folder 'bctc' (báo cáo tài chính)
  df.to_csv(csv_path, index = False)

## 4/ Lấy data hàng loạt

In [ ]:
# tính số lượng mã CK sẽ lấy để dễ theo dõi
num = len(mck_list)

# tạo loop xử lý hàng loạt:
for id, mck in enumerate(mck_list, 1):
  get_mck_csv(mck)
  print(f'MA {mck} ({id}/{num}) DA XONG !')
  x = randint(0,10)
  print(f'Tam nghi {x} giay.\n')
  sleep(x)

# tắt browser ảo sau khi chạy xong
browser.quit()

MA AAA (1/394) DA XONG !
Tam nghi 5 giay.

MA AAM (2/394) DA XONG !
Tam nghi 5 giay.

MA AAT (3/394) DA XONG !
Tam nghi 3 giay.

MA ABR (4/394) DA XONG !
Tam nghi 7 giay.

MA ABS (5/394) DA XONG !
Tam nghi 7 giay.

MA ABT (6/394) DA XONG !
Tam nghi 1 giay.

MA ACB (7/394) DA XONG !
Tam nghi 0 giay.

MA ACC (8/394) DA XONG !
Tam nghi 5 giay.

MA ACG (9/394) DA XONG !
Tam nghi 4 giay.

MA ACL (10/394) DA XONG !
Tam nghi 3 giay.

MA ADG (11/394) DA XONG !
Tam nghi 1 giay.

MA ADP (12/394) DA XONG !
Tam nghi 9 giay.

MA ADS (13/394) DA XONG !
Tam nghi 0 giay.

MA AGG (14/394) DA XONG !
Tam nghi 1 giay.

MA AGM (15/394) DA XONG !
Tam nghi 3 giay.

MA AGR (16/394) DA XONG !
Tam nghi 10 giay.

MA ANV (17/394) DA XONG !
Tam nghi 6 giay.

MA APC (18/394) DA XONG !
Tam nghi 0 giay.

MA APG (19/394) DA XONG !
Tam nghi 10 giay.

MA APH (20/394) DA XONG !
Tam nghi 4 giay.

MA ASG (21/394) DA XONG !
Tam nghi 5 giay.

MA ASM (22/394) DA XONG !
Tam nghi 3 giay.

MA ASP (23/394) DA XONG !
Tam nghi 4 gi

In [ ]:
# nen folder thanh file zip
!zip -r /content/bctc.zip /content/bctc

# import thu vien xu ly file
from google.colab import files

# tai file ve laptop
files.download('/content/bctc.zip')

  adding: content/bctc/ (stored 0%)
  adding: content/bctc/HNG.csv (deflated 52%)
  adding: content/bctc/APG.csv (deflated 54%)
  adding: content/bctc/CCL.csv (deflated 51%)
  adding: content/bctc/VGC.csv (deflated 50%)
  adding: content/bctc/CSM.csv (deflated 52%)
  adding: content/bctc/CTD.csv (deflated 51%)
  adding: content/bctc/HHS.csv (deflated 49%)
  adding: content/bctc/TTF.csv (deflated 51%)
  adding: content/bctc/CLC.csv (deflated 52%)
  adding: content/bctc/HDB.csv (deflated 50%)
  adding: content/bctc/PTC.csv (deflated 50%)
  adding: content/bctc/VSC.csv (deflated 49%)
  adding: content/bctc/KHG.csv (deflated 52%)
  adding: content/bctc/DLG.csv (deflated 50%)
  adding: content/bctc/L10.csv (deflated 52%)
  adding: content/bctc/TVT.csv (deflated 51%)
  adding: content/bctc/HCM.csv (deflated 54%)
  adding: content/bctc/VPB.csv (deflated 50%)
  adding: content/bctc/VHC.csv (deflated 49%)
  adding: content/bctc/RDP.csv (deflated 50%)
  adding: content/bctc/CLL.csv (deflated 51%

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#IV/ XỬ LÝ DATA RAW

## 1/ Kiểm tra tính đồng nhất cấu trúc table trong các file data raw
Phần này so sánh các table trong file data raw về các yếu tố:
1. Header.
2. Index.
3. Số column.
4. Số row.

In [ ]:
# khởi tạo list tổng hợp các index khác nhau (cùng 1 mã ck để đại diện) của các file data raw
data_indices = []
mck_indices = []

# khởi tạo dict tổng hợp với key là các header khác nhau (và value là 1 mã ck đại diện) của các file data raw
data_headers = {}
# mck_headers = []

# khởi tạo 1 series trống để join với các index khác nhau để quan sát trực quan
srs = pd.Series()

# tao vòng lặp lấy các index & header khác nhau
for mck in mck_list:
 data_path = '/content/bctc/' + mck + '.csv'
 data = pd.read_csv(data_path)
 index = list(data['0'])
 header = list(data.iloc[0])

 #  kiển tra index
 if index not in data_indices:
  mck_indices.append(mck) # show đại diện các mã đầu tiên có file data với index khác nhau
  data_indices.append(index)
  srs = pd.concat([srs,data['0']], axis=1) # nối các index khác nhau vào cùng 1 series

  # kiểm tra header
 if header not in data_headers.values():
  data_headers.update({mck:header})

# hiển thị kết quả để đối chiếu
for k,v in data_headers.items():
  print(k,'\n',v,'\n')
display(mck_indices)
display(srs)

<ipython-input-3-d3cee5b90a4c>:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  srs = pd.Series()


AAA 
 ['KẾT QUẢ KINH DOANH', 'Quý 4/2021', 'Quý 1/2022', 'Quý 2/2022', 'Quý 3/2022', 'Quý 4/2022', 'Quý 1/2023', 'Quý 2/2023', 'Quý 3/2023'] 

AGG 
 ['KẾT QUẢ KINH DOANH', 'Quý 3/2021', 'Quý 4/2021', 'Quý 1/2022', 'Quý 2/2022', 'Quý 3/2022', 'Quý 4/2022', 'Quý 1/2023', 'Quý 2/2023'] 

FIR 
 ['KẾT QUẢ KINH DOANH', 'Quý 1/2022', 'Quý 2/2022', 'Quý 3/2022', 'Quý 4/2022', 'Quý 1/2023', 'Quý 2/2023', 'Quý 3/2023', 'Quý 4/2023'] 

HID 
 ['KẾT QUẢ KINH DOANH', 'Quý 2/2021', 'Quý 3/2021', 'Quý 4/2021', 'Quý 1/2022', 'Quý 2/2022', 'Quý 3/2022', 'Quý 4/2022', 'Quý 1/2023'] 

IBC 
 ['KẾT QUẢ KINH DOANH', 'Quý 1/2021', 'Quý 2/2021', 'Quý 3/2021', 'Quý 4/2021', 'Quý 1/2022', 'Quý 2/2022', 'Quý 3/2022', 'Quý 4/2022'] 

JVC 
 ['KẾT QUẢ KINH DOANH', 'Quý 2/2021', 'Quý 3/2021', 'Quý 4/2021', 'Quý 1/2022', 'Quý 2/2022', 'Quý 3/2022', 'Quý 1/2023', 'Quý 2/2023'] 



['AAA', 'ACB', 'AGR']

,0,0,0,0
0,NaN,KẾT QUẢ KINH DOANH,KẾT QUẢ KINH DOANH,KẾT QUẢ KINH DOANH
1,NaN,Doanh thu thuần,Tổng thu nhập,DT từ KD chứng khoán
2,NaN,LN gộp,Tổng chi phí,LN gộp
3,NaN,LN thuần từ HĐKD,Lợi nhuận trước thuế,LNT từ KD chứng khoán
4,NaN,LNST thu nhập DN,Lợi nhuận sau thuế,LNST thu nhập DN
5,NaN,LNST của CĐ cty mẹ,LNST của CĐ cty mẹ,LNST của CĐ cty mẹ
6,NaN,CÂN ĐỐI KẾ TOÁN,CÂN ĐỐI KẾ TOÁN,CÂN ĐỐI KẾ TOÁN
7,NaN,Tài sản ngắn hạn,Tổng tài sản Có,Tài sản ngắn hạn
8,NaN,Tổng tài sản,- Cho vay khách hàng,Tổng tài sản
9,NaN,Nợ ngắn hạn,- Các khoản đầu tư,Nợ ngắn hạn


> Từ kết quả trên có thể thấy:
1. Các table trong các file data có cùng số column (9) nhưng có 6 "header" khác nhau, trong đó 5 header có các Quý là liên tục còn header thứ 6 (Vd mã JVC) gián đoạn >> bỏ các mã CK bị gián đoạn.
2. Các table có cùng số row (20), tuy nhiên nội dung index lại khác nhau cho 3 nhóm: Cty `Thường` vs `Ngân Hàng` vs `Chứng Khoán` >> chú ý khi chọn Row về sau.

## 2/ Xử lý data raw và tổng hợp
Dựa vào kết quả đối chiếu phần trước để tiến hành xử lý data raw và lấy ra các dữ liệu cần thiết, sau đó tổng hợp lại thành 1 file data clean để làm dữ liệu đầu vào cho quá trình Machine Learning.

### 2.1/ Xem xét sample

In [ ]:
# Xem xét 1 table của mã CK đầu tiên
df1 = pd.read_csv('/content/bctc/AAA.csv')
df1

,0,2_x,3_x,4_x,5_x,2_y,3_y,4_y,5_y
0,KẾT QUẢ KINH DOANH,Quý 4/2021,Quý 1/2022,Quý 2/2022,Quý 3/2022,Quý 4/2022,Quý 1/2023,Quý 2/2023,Quý 3/2023
1,Doanh thu thuần,"4,198,836","4,027,684","4,612,210","3,280,335","3,406,559","3,616,713","2,791,313","3,529,079"
2,LN gộp,"380,689","393,676","343,030","308,188","34,256","246,170","221,721","319,749"
3,LN thuần từ HĐKD,"94,399","122,182","115,483","107,594","-161,340","76,298","63,287","123,460"
4,LNST thu nhập DN,"69,876","96,488","92,405","76,617","-152,754","63,797","50,791","103,547"
5,LNST của CĐ cty mẹ,"60,518","88,086","84,623","79,866","-97,726","53,895","42,970","99,050"
6,CÂN ĐỐI KẾ TOÁN,Quý 4/2021,Quý 1/2022,Quý 2/2022,Quý 3/2022,Quý 4/2022,Quý 1/2023,Quý 2/2023,Quý 3/2023
7,Tài sản ngắn hạn,"5,336,716","5,562,194","5,692,503","6,138,747","5,623,485","5,674,658","5,568,006","5,858,656"
8,Tổng tài sản,"9,991,632","10,477,077","10,928,358","11,277,995","10,781,316","10,901,971","11,231,810","11,610,262"
9,Nợ ngắn hạn,"3,260,485","3,838,442","4,002,484","3,326,091","3,195,249","3,265,393","3,597,293","3,206,292"


In [ ]:
# xem thông tin chung
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       20 non-null     object
 1   2_x     20 non-null     object
 2   3_x     20 non-null     object
 3   4_x     20 non-null     object
 4   5_x     20 non-null     object
 5   2_y     20 non-null     object
 6   3_y     20 non-null     object
 7   4_y     20 non-null     object
 8   5_y     20 non-null     object
dtypes: object(9)
memory usage: 1.5+ KB


> Nhận xét:
- Các row 0, 6, 13 chứa thông tin trùng lặp.
- Một số row (vd ROS, ROA, ROE) hoàn toàn tính dựa vào 1 trong số các row từ 1 > 11.
- Các cột dữ liệu số đang ở dạng `text` >> xử lý chuyến sang dạng số `float`.  


### 2.2/ Lên các bước xử lý
Từ nhận xét và trong thực tế giữa các Công ty có số liệu chênh lệch nhau rất lớn nên chúng ta có nhu cầu chọn một số Row và tạo ra một số Fearure để thuận tiện cho việc so sánh.
Theo đó có các bước xử lý:
1. Giữ lại row 0 làm header và bỏ các row 6, 13.
2. Xử lý `text` chuyến sang `float`.
3. Chọn các row *phù hợp* để tạo nên các `Feature` của data tổng hợp sao cho đảm bảo tính *Khách quan* và *Toàn diện*.

4. Biến đổi table ban đầu thành 1 table chỉ có 1 row và các Feature mới dựa trên các row vừa chọn.

#### Chọn ROW

Chọn các row có tính chất sau để tạo nên các `Feature` của data tổng hợp nhằm Đảm bảo tính **Khách quan** và **Toàn diện**:
 - **Độc lập**: không là kết quả tính toán hoàn toàn dựa trên các row khác.
 - **Đại diện**: có đủ 3 nhóm số liệu từ 2 phía Doanh Nghiệp và Thị Trường.
> Cụ thể:

Nguồn        | Nhóm số liệu             | Tên Row                | Ý nghĩa
-------------|--------------------------|------------------------|-------------
Doanh Nghiệp | Hiệu quả Kinh doanh      |Doanh thu thuần/ Tổng thu nhập/ DT từ KD chứng khoán | Tổng doanh thu của DN: càng tăng càng tốt.
             |                          |LNST của CĐ cty mẹ | Lợi nhuận thực tế DN: càng tăng càng tốt.
             | Cơ cấu nguồn vốn         | Nợ phải trả | Tổng nợ DN: càng giảm càng tốt.
             |                          | Vốn chủ sở hữu/ Vốn của TCTD | Tổng vốn DN: ổn định hoặc tăng thì tốt.
Thị Trường   | Chỉ số thị trường        |EPS | Thu nhập mỗi CP: càng tăng càng tốt.
             |                          |BVPS | Giá trị sổ sách: càng tăng càng tốt.
             |                          |P/E | Tỷ lệ giá CP/Thu nhập: càng tăng càng tốt.


#### Tạo FEATURE

Biến đổi table ban đầu thành 1 table chỉ có 1 row và các Feature mới như sau:
  - Với 2 chỉ số Doanh thu, Lợi nhuận: từ giá trị mỗi chỉ số của mỗi Quý trong 8 Quý liên tiếp từ Q4 2021 đến Q3 2023 tính ra `4 column` là **Tỷ lệ tăng trưởng của 4 Quý gần nhất (QGN) so với cùng kỳ năm ngoái (YoY)** và `3 column` là **Tỷ lệ tăng trưởng của 3 Quý gần nhất so với Quý đầu kỳ**.
  > Ví dụ: 1 chỉ số Doanh thu với dữ liệu của 8 Quý sẽ tạo ra 7 cột Feature mới lần lượt là:
    1. Tăng trưởng Quý gần nhất (Q3 2023) so với cùng kỳ năm ngoái (Q3 2022): `TtDT_QG1_YoY`.
    2. Tăng trưởng Quý gần thứ 2 (Q2 2023) so với cùng kỳ năm ngoái (Q2 2022): `TtDT_QG2_YoY`.
    3. Tăng trưởng Quý gần thứ 3 (Q1 2023) so với cùng kỳ năm ngoái (Q1 2022): `TtDT_QG3_YoY`.
    4. Tăng trưởng Quý gần thứ 4 (Q4 2022) so với cùng kỳ năm ngoái (Q4 2021):  `TtDT_QG4_YoY`.
    5. Tăng trưởng Quý gần nhất (Q3 2023) so với Quý đầu năm (Q4 2022): `TtDT_QG1`.
    6. Tăng trưởng Quý gần thứ 2 (Q2 2023) so với Quý đầu năm (Q4 2022): `TtDT_QG2`.
    7. Tăng trưởng Quý gần thứ 3 (Q1 2023) so với Quý đầu năm (Q4 2022): `TtDT_QG3`.

    > Mục đích: theo dõi sự tăng trưởng của 2 chỉ số này trong 1 năm gần đây và đối chiếu cùng kỳ cả năm trước đó để cân nhắc nếu có yếu tố mùa vụ của một vài Ngành (Vd Du lịch, Buôn bán hàng tiêu dùng) gây ảnh hưởng Doanh thu & Lợi nhuận.

    > Với chỉ số còn lại thực hiện tương tự (Lợi nhuận: LN). Tổng lại *2 Chỉ số x 8 Quý = 14 Feature mới*.

  - Với 5 chỉ số Tổng nợ (TN), Tổng vốn (TV), EPS, BVPS, P/E: mỗi chỉ số lấy giá trị của 4 Quý gần nhất Q4 2022 > Quý 3 2023 để tạo thành `3 column` là **Tỷ lệ tăng trưởng của 3 Quý gần nhất so với Quý đầu kỳ**.
   > Vd: Với 1 chỉ số EPS và giá trị trong 4 Quý gần nhất cho ra 3 column:
    1. Tăng trưởng EPS trong Quý gần nhất (Q3 2023) so với Quý đầu kỳ (Q4 2022): `TtEPS_QG1`.
    2. Tăng trưởng EPS trong Quý gần thứ 2 (Q2 2023) so với Quý đầu kỳ (Q4 2022): `TtEPS_QG2`.
    3. Tăng trưởng EPS trong Quý gần thứ 3 (Q1 2023) so với Quý đầu kỳ (Q4 2022): `TtEPS_QG3`.
    
    > Mục đích: theo dõi sự tăng trưởng của 5 chỉ số này trong 1 năm gần đây.

    > Với 4 chỉ số còn lại thực hiện tương tự. Tổng lại *5 Chỉ số x 4 Quý = 15 Feature mới*.
    

> Như vậy từ **1 Table** ban đầu có `20 Row x 9 Column` của 1 mã CK chúng ta sẽ chuyển thành **1 Record** có `1 Row x 30 Column`  (1 Column mã CK + 14 phía trên + 15 phía trên) trong dataFrame tổng hợp mới.

### 2.3/ Xử lý và tổng hợp

#### Loại bỏ các mã CK có dữ liệu gián đoạn

In [ ]:
# kiểm tra các mã CK có data với index bị gián đoạn
gd_idx = ['KẾT QUẢ KINH DOANH', 'Quý 2/2021', 'Quý 3/2021', 'Quý 4/2021', 'Quý 1/2022', 'Quý 2/2022', 'Quý 3/2022', 'Quý 1/2023', 'Quý 2/2023']
mgd = []

# tạo vòng lặp tìm các mã gián đoạn
for mck in mck_list:
 data_path = '/content/bctc/' + mck + '.csv'
 data = pd.read_csv(data_path)
 header = list(data.iloc[0])
 if header == gd_idx:
  mgd.append(mck)

print(len(mgd))
mgd

1


['JVC']

In [ ]:
# loại bỏ các mã này bằng loop
for m in mgd:
  mck_list.remove(m)

# kiểm tra
len(mck_list)

393

#### Tạo header và row của DataFrame tổng hợp

In [ ]:
# khởi tạo header: các column tăng trưởng YoY đặt trước và Quý trong năm để sau >>> dễ loop
record_header = ['Mã',
              'TtDT_QG4_YoY','TtDT_QG3_YoY','TtDT_QG2_YoY','TtDT_QG1_YoY',
              'TtLN_QG4_YoY','TtLN_QG3_YoY','TtLN_QG2_YoY','TtLN_QG1_YoY',
              'TtDT_QG3','TtDT_QG2','TtDT_QG1',
              'TtLN_QG3','TtLN_QG2','TtLN_QG1',
              'TtTN_QG3','TtTN_QG2','TtTN_QG1',
              'TtTV_QG3','TtTV_QG2','TtTV_QG1',
              'TtEPS_QG3','TtEPS_QG2','TtEPS_QG1',
              'TtBVPS_QG3','TtBVPS_QG2','TtBVPS_QG1',
              'TtPE_QG3','TtPE_QG2','TtPE_QG1']

#### Tạo record với các Feature mới

In [ ]:
# khởi tạo biến chứa dữ liệu của DataFrame tổng hợp
record_values = []

# tạo loop để xử lý từng mã Ck trong Ds
for mck in mck_list:
 data_path = '/content/bctc/' + mck + '.csv'
 df = pd.read_csv(data_path)

 # lấy ra các row cần thiết, chú ý nhóm Ngân Hàng có row 11 là "- Tiền gửi (Khách hàng+TCTD)"
 df.columns = df.iloc[0]
 if df['KẾT QUẢ KINH DOANH'][11] == 'Vốn chủ sở hữu':
  df = df.iloc[[1, 5, 10, 11, 17, 18, 19]]
 else:
  df = df.iloc[[1, 5, 10, 12, 17, 18, 19]] # lấy row 12 thay vì 11 tức lấy 'Vốn của TCTD' của nhóm Ngân Hàng

 # reset lại index
 df.reset_index(inplace=True, drop=True)

 # xử lý thay các ô null dạng text "-" hay "--" bằng "NaN"
 df = df.replace(dict.fromkeys(['-','--'], np.nan))

 # loại bỏ "," trong value dạng số và đổi type thành 'float'
 for col in df.columns[1:]:
  df[col] = df[col].str.replace(',','').astype(float)

#  for i in range(1,9):
#   df.iloc[:,i] = df.iloc[:,i].str.replace(',','').astype(float)

 # tạo loop kiểm từng row xem có null
 for i in range(7):
  if df.iloc[i].isnull().sum()>0:
    m = df.iloc[i][1:].mean() # tính giá trị trung bình của row này
    for k in range(9):
      if pd.isna(df.loc[i][k]): # kiểm tra từng cell xem có null
        df.loc[i, df.columns[k]] = m # thay null bằng giá trị trung bình

 # lưu thông tin đầu tiên là Mã CK vào column đầu tiên là "Mã" vào biến chứa nội dung 1 record
 record = [mck]

 # tạo loop tính 4 giá trị Tt YoY của 4 Quý gần nhất cho 2 chỉ số Doanh thu & Lợi nhuận
 for i in range(2):
  for k in range(4):
    value = (df.iloc[i][k+5] - df.iloc[i][k+1])*100/df.iloc[i][k+1]
    record.append(value) # lưu các giá trị vào record

 # tạo loop tính 3 giá trị Tt của 3 Quý gần nhất cho 5 chỉ số Nợ, Vốn, EPS, BVPS, E/P
 for i in range(7):
  for k in range(3):
    value = (df.iloc[i][k+6] - df.iloc[i][5])*100/df.iloc[i][5]
    record.append(value) # lưu các giá trị vào record
 # thêm record vào danh sách record ban đầu
 record_values.append(record)

# kiểm tra
len(record_values)

393

#### Lưu kết quả vào DataFrame & kiểm tra

In [ ]:
# lưu vào dataFrame
finance_record = pd.DataFrame(record_values, columns=record_header)

# xem 10 row ngẫu nhiên
display(finance_record.sample(10))

# xem 10 row cuối cùng
display(finance_record.tail(10))

,Mã,TtDT_QG4_YoY,TtDT_QG3_YoY,TtDT_QG2_YoY,TtDT_QG1_YoY,TtLN_QG4_YoY,TtLN_QG3_YoY,TtLN_QG2_YoY,TtLN_QG1_YoY,TtDT_QG3,...,TtTV_QG1,TtEPS_QG3,TtEPS_QG2,TtEPS_QG1,TtBVPS_QG3,TtBVPS_QG2,TtBVPS_QG1,TtPE_QG3,TtPE_QG2,TtPE_QG1
95,DRC,-16.44,-13.31,1.19,-16.95,-6.30,-61.39,-39.30,-1.84,-0.13,...,-4.95,-13.09,-23.75,-24.21,-1.85,-8.90,-4.94,22.32,39.72,47.04
92,DPM,-23.33,-43.99,-26.05,-17.23,-30.21,-87.70,-92.12,-93.58,-16.28,...,-17.89,-33.47,-54.76,-71.62,-9.72,-9.60,-17.89,16.83,79.21,202.97
340,TRC,19.25,-11.70,-27.78,37.55,-12.05,-49.57,-75.74,2.28,-41.76,...,1.91,-3.46,-33.56,-33.20,0.04,-1.00,1.91,-13.38,45.60,57.99
168,IDI,21.42,-5.86,-23.44,-10.94,-74.75,-92.68,-90.17,-79.61,3.13,...,1.99,-34.12,-71.58,-85.64,0.64,1.29,1.99,55.43,315.22,760.87
57,CMV,5.55,-7.34,-8.68,-1.75,-74.56,-52.88,-57.39,158.43,-14.41,...,-4.19,-22.09,-46.99,-37.55,1.22,-5.16,-4.19,14.23,64.60,39.78
204,MSN,-13.37,2.84,4.34,3.24,-93.05,-86.53,-89.27,-91.09,-9.38,...,3.92,-41.17,-64.99,-78.23,1.28,2.42,3.40,42.19,130.86,276.74
81,DCM,14.12,-32.89,-19.41,-8.97,-8.57,-84.91,-72.22,-89.87,-38.66,...,-9.91,-30.04,-47.56,-62.83,1.79,4.32,-9.91,28.96,89.63,232.01
110,EVG,-0.09,5.11,-19.54,-20.22,-71.43,-9.68,111.91,-8.07,2.52,...,1.13,-9.09,32.23,29.75,0.16,0.89,1.13,8.29,29.98,30.72
235,PGV,38.49,32.80,4.99,29.12,-66.48,10.17,-26.82,167.16,1.76,...,10.34,2.70,-8.01,24.13,0.51,5.18,10.34,-23.38,-10.91,-12.80
299,SVC,201.13,30.31,13.61,-14.15,-430.68,10.11,-94.56,-93.91,39.94,...,5.31,2.37,-29.62,-59.80,7.49,-46.74,-47.34,-46.73,-48.94,-1.84


,Mã,TtDT_QG4_YoY,TtDT_QG3_YoY,TtDT_QG2_YoY,TtDT_QG1_YoY,TtLN_QG4_YoY,TtLN_QG3_YoY,TtLN_QG2_YoY,TtLN_QG1_YoY,TtDT_QG3,...,TtTV_QG1,TtEPS_QG3,TtEPS_QG2,TtEPS_QG1,TtBVPS_QG3,TtBVPS_QG2,TtBVPS_QG1,TtPE_QG3,TtPE_QG2,TtPE_QG1
383,VPS,15.60,-24.13,-3.20,-1.27,151.15,-95.75,70.92,-16.32,-66.68,...,-2.62,-12.20,-7.52,-11.35,-0.22,-3.73,-2.62,25.38,35.38,27.73
384,VRC,41.80,-5.76,2.84,39.46,30177.78,-57.14,-35.59,-85.09,-32.00,...,0.02,-0.29,-0.29,-2.94,0.00,0.01,0.02,-0.94,13.35,8.97
385,VRE,52.48,41.90,17.48,66.20,549.34,171.26,29.45,65.95,-6.77,...,10.11,23.59,31.89,51.00,3.17,6.17,10.12,-9.11,-22.79,-34.32
386,VSC,2.64,-1.28,4.50,9.83,-41.65,-67.96,-78.68,-59.57,-10.89,...,0.67,-22.54,-46.05,-60.63,-4.99,-0.85,-8.48,25.55,117.25,140.22
387,VSH,51.81,10.34,-0.58,-41.40,98.26,18.02,1.86,-88.40,-7.34,...,0.96,5.76,6.13,-9.29,9.35,0.46,0.97,17.26,31.98,51.78
388,VSI,119.14,-8.52,54.04,113.88,-16.19,-24.15,24.72,33.55,-66.78,...,2.49,-6.11,1.43,9.69,2.34,-1.50,2.49,3.59,-11.68,-14.37
389,VTB,7.01,-56.34,-61.13,-40.17,367.27,-43.91,43.84,7.98,-72.91,...,0.52,-6.71,-1.01,0.00,-0.01,-0.66,0.52,-15.60,-20.20,-28.28
390,VTO,70.72,10.00,-3.14,-21.70,-38.99,629.03,262.90,-38.97,-19.45,...,-4.00,14.18,22.13,11.02,1.40,-4.98,-4.00,12.27,-0.86,14.27
391,YBM,-29.91,-19.96,4.49,23.95,-50.72,-43.79,22.12,10.25,35.18,...,5.26,-22.25,-18.06,-16.20,0.95,3.43,5.26,2.91,5.52,22.09
392,YEG,-54.14,-10.18,33.01,57.70,-99.22,374.31,-14.64,-37.09,-46.60,...,50.47,16.46,11.13,-13.01,0.09,0.70,-38.30,-3.72,41.48,54.23


In [ ]:
# check null
print(finance_record.isnull().sum().sum())

# xem row có null
finance_record[finance_record.isna().any(axis=1)]

12


,Mã,TtDT_QG4_YoY,TtDT_QG3_YoY,TtDT_QG2_YoY,TtDT_QG1_YoY,TtLN_QG4_YoY,TtLN_QG3_YoY,TtLN_QG2_YoY,TtLN_QG1_YoY,TtDT_QG3,...,TtTV_QG1,TtEPS_QG3,TtEPS_QG2,TtEPS_QG1,TtBVPS_QG3,TtBVPS_QG2,TtBVPS_QG1,TtPE_QG3,TtPE_QG2,TtPE_QG1
17,APC,-19.14,-38.15,-27.32,-8.14,133.15,524.76,-584.43,-394.40,-30.98,...,-4.74,142.16,303.09,338.41,-2.44,-4.36,-4.74,NaN,NaN,NaN
149,HNG,-50.49,-40.64,2.39,-16.32,242.41,-0.17,-76.98,-52.14,-16.37,...,-4.79,-0.00,-11.06,-16.63,-4.19,-8.23,-4.81,NaN,NaN,NaN
164,HVN,346.76,112.13,102.19,12.23,-22.14,133.77,-96.03,-47.00,-7.97,...,54.43,11.80,-15.00,-27.89,35.79,36.32,54.42,NaN,NaN,NaN
268,SCD,15.06,27.99,-97.12,-5.79,282.47,-21.46,256.16,59.96,28.68,...,-68.34,-1.57,50.55,77.80,-2.60,-35.41,-68.34,NaN,NaN,NaN


In [ ]:
# bỏ các row có null
finance_record = finance_record.drop(index=[17,149,164,268])

# kiểm tra
len(finance_record)

389

In [ ]:
# reset lại index
finance_record.reset_index(inplace=True, drop=True)

# kiểm tra
finance_record.iloc[[17,149,164,268],:]

,Mã,TtDT_QG4_YoY,TtDT_QG3_YoY,TtDT_QG2_YoY,TtDT_QG1_YoY,TtLN_QG4_YoY,TtLN_QG3_YoY,TtLN_QG2_YoY,TtLN_QG1_YoY,TtDT_QG3,...,TtTV_QG1,TtEPS_QG3,TtEPS_QG2,TtEPS_QG1,TtBVPS_QG3,TtBVPS_QG2,TtBVPS_QG1,TtPE_QG3,TtPE_QG2,TtPE_QG1
17,APG,-98.82,-57.06,84.20,5.20,-183.88,-45.53,-185.52,-86.88,1438.76,...,11.16,14.95,-52.53,-72.80,3.94,8.14,5.86,0.00,0.00,0.00
149,HPX,-15.95,166.81,42.16,-58.50,-86.76,-233.02,636.71,-96.00,-54.33,...,-3.69,-33.83,14.89,-47.66,-6.08,-3.81,-3.69,34.05,-15.85,126.79
164,ICT,-32.66,-61.00,-78.08,78.34,-64.65,-87.94,-816.62,1640.60,-29.60,...,-4.76,-61.98,-147.43,-72.90,0.33,-2.66,-4.76,114.10,22.73,174.46
268,SFG,-70.27,-44.93,-38.52,-26.30,-142.58,19.83,-138.59,738.84,48.47,...,3.45,12.46,-53.78,-16.29,5.57,0.32,3.45,-23.98,101.49,17.15


> Row 17 mã APC có null ban đầu giờ đã được thay bởi mã APG không có null.

> Tương tự thì 149: HNG > HPX, 164:	HVN > ICT, 268:	SCD > SFG.

#### Lưu kết quả vào csv & tải về

In [ ]:
# join dataFrame này với Ds Công ty ban đầu (file B1)
finance_record = pd.merge(companies, finance_record , on = 'Mã')

# kiểm tra
finance_record.head(5)

,Mã,Công ty,Ngành,Khối lượng NY/ĐKGD,TtDT_QG4_YoY,TtDT_QG3_YoY,TtDT_QG2_YoY,TtDT_QG1_YoY,TtLN_QG4_YoY,TtLN_QG3_YoY,...,TtTV_QG1,TtEPS_QG3,TtEPS_QG2,TtEPS_QG1,TtBVPS_QG3,TtBVPS_QG2,TtBVPS_QG1,TtPE_QG3,TtPE_QG2,TtPE_QG1
0,AAA,Công ty cổ phần Nhựa An Phát Xanh,"Nhựa, cao su & sợi",382274496,-18.87,-10.20,-39.48,7.58,-261.48,-38.82,...,4.54,-24.83,-52.62,-41.46,1.03,2.88,4.54,80.99,233.46,133.53
1,AAM,Công ty Cổ phần Thủy sản Mekong,Nuôi trồng nông & hải sản,12346411,-30.90,-45.60,-40.07,-39.19,-22.45,-41.00,...,-3.08,-5.07,-52.13,-73.84,0.51,-2.58,-3.08,5.79,93.50,250.99
2,AAT,Công ty cổ phần Tiên Sơn Thanh Hoá,Hàng May mặc,70819103,296.41,9.97,-64.84,-32.35,28.54,6876.19,...,-0.87,1.55,-94.35,-94.42,-1.07,-0.53,-10.69,-1.14,2154.57,1780.86
3,ABR,Công ty Cổ phần Đầu tư Nhãn hiệu Việt,Tư vấn & Hỗ trợ KD,20000000,24.13,19.06,-17.76,-77.11,56.75,56.35,...,10.69,11.16,44.18,34.45,3.29,8.67,10.69,-7.04,5.86,6.70
4,ABS,Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận,Phân phối hàng chuyên dụng,80000000,35.56,-76.45,13.50,-14.80,-81.70,177.19,...,-33.55,23.08,188.17,172.49,-37.48,-33.70,-33.55,-11.27,-54.47,-53.37


In [ ]:
# lưu vào file csv
finance_record.to_csv("/content/B2_finance_record.csv", index = False, encoding='utf-8')

# import thư viện xử lý file
from google.colab import files

# tải file csv về laptop
files.download('B2_finance_record.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>